# DBSCAN 介绍和步骤

DBSCAN （Density-Based Spatial[空间的] Clustering of Applications with Noise）是密度聚类算法

核心思想是：基于 “密度” 划分聚类 —— 把空间中密度足够高的相邻点归为一类，同时标记低密度的孤立点为 “噪声”。

相比 K-Means（需指定聚类数、仅适用于凸聚类），DBSCAN 无需预设聚类数量，能发现任意形状的聚类（如环形、不规则形），还能自动过滤噪声，特别适配机器人感知中的点云聚类（如 LiDAR 点云分割行人 / 车辆）、车道线点聚类去噪等场景。

## 二、DBSCAN 核心概念（理解算法的前提）

在讲解流程前，先明确 4 个核心定义（以二维 / 三维点集为例，对应机器人感知的图像特征点 / 点云）：

｜ 概念 ｜ 定义 ｜ 物理意义 ｜
｜ ---- ｜ ---- ｜ ---- ｜
｜ ε（Epsilon，邻域半径） ｜ 以某个点为中心，半径 ε 的圆形 / 球形区域（ε 邻域） ｜ 点云聚类中，ε=0.5m 表示 “距离某 LiDAR 点 0.5m 内的所有点视为相邻” ｜
｜ MinPts（最小点数） ｜ ε 邻域内至少包含的点数（含自身） ｜ 点云聚类中，核心点是某物体（如行人）的 “主体点”，是聚类的核心 ｜
｜ 核心点（Core Point） ｜ ε 邻域内点数 ≥ MinPts 的点 ｜ 点云聚类中，核心点是某物体（如行人）的 “主体点”，是聚类的核心 ｜
｜ 边界点（Border Point） ｜ ε 邻域内点数 < MinPts，但落在某个核心点的 ε 邻域内 ｜ 物体边缘的点（如行人手臂末端的点），依附核心点归为一类 ｜
｜ 噪声点（Noise Point） ｜ 既不是核心点，也不在任何核心点的 ε 邻域内 ｜ 点云中的杂点（如地面反光点、传感器噪声），直接过滤 ｜
｜ 密度可达 ｜ 点 A→B→C…→D，若每一步的点都是核心点，且相邻点在 ε 邻域内，则 D 从 A 密度可达 ｜ 同一物体的点之间 “密度可达”，不同物体的点不可达 ｜

## 三、DBSCAN 完整算法流程（分步拆解）

以机器人感知中 “LiDAR 点云聚类” 为例，流程如下（伪代码 + 通俗解释）：

### 步骤 1：初始化参数与数据

+ 输入：点集 P（如 LiDAR 点云的 XYZ 坐标）、ε、MinPts；
+ 输出：聚类标签（每个点对应 “聚类 ID” 或 “噪声”）；
+ 初始化：所有点标记为 “未访问”，聚类 ID 初始化为 0。

### 步骤 2：遍历所有未访问的点

对每个未访问的点 $p \in P$：

1. 标记 p 为 “已访问”；
2. 找到 p 的 ε 邻域内所有点，记为 $N(p)$；
3. 若 $|N(p)| < MinPts$ → 标记 p 为 “噪声点”，跳过；
4. 若 $|N(p)| ≥ MinPts$ → p 是核心点，开始构建聚类：
    + 创建新聚类（聚类 ID+1），将 p 加入该聚类；
    + 初始化队列 Q，将 $N(p)$ 中所有未访问的点加入 Q；
    + 遍历队列 Q 中的每个点 q：
        + 标记 q 为 “已访问”；
        + 找到 q 的 ε 邻域内所有点 $N(q)$；
        + 若 $|N(q)| ≥ MinPts$（q 也是核心点）→ 将 $N(q)$ 中未访问、未聚类的点加入 Q；
        + 将 q 加入当前聚类（无论 q 是核心点 / 边界点）；
    + 队列遍历结束，当前聚类构建完成。

### 步骤 3：输出结果

+ 所有点遍历完成后，每个点会被标记为：某聚类 ID（如 1、2、3）或 “噪声”；
+ 机器人感知中：聚类 ID 对应不同物体（如 ID1 = 行人、ID2 = 车辆），噪声点直接过滤。